## Imports

In [24]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import gc
import sys
import warnings

from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import roc_auc_score, log_loss
from scipy.special import logit
from scipy.stats import norm

pd.set_option("display.float_format", lambda x: "%.3f" % x)
warnings.filterwarnings("ignore")

In [6]:
from google.colab import drive

drive.mount("/content/drive/")

sys.path.append("/content/drive/My Drive/hse/recsys")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Functions

In [7]:
def load_data(data_path: str) -> pd.DataFrame:
    columns_to_use = [
        "date_time",
        "oaid_hash",
        "zone_id",
        "banner_id",
        "os_id",
        "country_id",
        "banner_id0",
        "banner_id1",
        "g0",
        "g1",
        "coeff_sum0",
        "coeff_sum1",
        "clicks",
    ]

    data = pd.read_csv(data_path, usecols=columns_to_use, parse_dates=["date_time"])
    return data

In [8]:
def get_interactions(data: pd.DataFrame, interactions_list: list[str]) -> pd.DataFrame:
    for col1, col2 in interactions_list:
        data[f"{col1};{col2}"] = data[col1].astype(str) + "_" + data[col2].astype(str)
    return data

In [9]:
def group_categories(
    data: pd.DataFrame, column_names: list[str], threshold: float
) -> pd.DataFrame:
    for column in column_names:
        new_value = data[column].max() + 1000
        categories_to_group = data[column].value_counts(normalize=True) < threshold
        categories_to_group = categories_to_group[categories_to_group].index
        data.loc[data[column].isin(categories_to_group), column] = new_value
    return data

In [25]:
def feature_engineering(data: pd.DataFrame) -> pd.DataFrame:
    # Удаление выброса
    data = data[data.date_time.dt.date != pd.Timestamp("2021-09-01").date()]

    # Добавление фичей: время суток и день недели
    data["hour"] = data.date_time.dt.hour
    data["weekday"] = data.date_time.dt.weekday

    # Объединение редких категорий
    data = group_categories(data, ["zone_id"], 0.001)
    oaid_hash_vc = data.oaid_hash.value_counts()
    oaid_hash_to_change = oaid_hash_vc[oaid_hash_vc <= 2].index
    data.loc[data.oaid_hash.isin(oaid_hash_to_change), "oaid_hash"] = -100

    return data

In [18]:
def create_dataset(data: pd.DataFrame, interactions: list[str]):
    print("Dataset preparation...\n")

    # Предобработка и добавление фичей
    data = feature_engineering(data)

    # Train / test split
    ts = pd.Timestamp("2021-10-02 00:00:00")
    data.sort_values(by="date_time", inplace=True)
    train_mask, test_mask = data.date_time < ts, data.date_time >= ts
    X_train, X_test = data[train_mask].copy(), data[test_mask].copy()

    # Удалим строки в тесте, где есть nan
    X_test = X_test.dropna()

    # Оставим только те строки в тесте, для которых banner_id == banner_id0
    X_test = X_test[X_test["banner_id"] == X_test["banner_id0"]]

    y_train, y_test = X_train.clicks.to_numpy(), X_test.clicks.to_numpy()

    # Нужно создать отдельный тестовый набор, где banner_id == banner_id1
    X_test1 = X_test.copy()
    X_test1["banner_id"] = X_test1["banner_id1"]

    # Сохраним данные для подсчета ips
    ips_data = X_test[["g0", "g1", "coeff_sum0", "coeff_sum1"]].copy()

    # Добавим интеракции в train, test, test1
    X_train = get_interactions(X_train, interactions)
    X_test = get_interactions(X_test, interactions)
    X_test1 = get_interactions(X_test1, interactions)

    # Удаляем лишние колонки перед one-hot-encoding
    columns_to_drop = ["clicks", "date_time", "g0", "g1", "coeff_sum0", "coeff_sum1"]
    X_train.drop(columns_to_drop, axis=1, inplace=True)
    X_test.drop(columns_to_drop, axis=1, inplace=True)
    X_test1.drop(columns_to_drop, axis=1, inplace=True)

    print(f"Train size: {X_train.shape[0]}")
    print(f"Test_size: {X_test.shape[0]}")

    # One hot encoding
    encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=True)
    X_train = encoder.fit_transform(X_train)
    X_test = encoder.transform(X_test)
    X_test1 = encoder.transform(X_test1)

    print(f"Features number: {X_train.shape[1]}")
    print("\nDataset is ready!")

    del data
    gc.collect()

    return (X_train, X_test, X_test1), (y_train, y_test, y_test), ips_data

## Preparing dataset and model

In [27]:
data_path = "/content/drive/My Drive/hse/recsys/data/data.csv"
data = load_data(data_path)

In [28]:
data.head()

,date_time,zone_id,banner_id,oaid_hash,os_id,country_id,banner_id0,g0,coeff_sum0,banner_id1,g1,coeff_sum1,clicks
0,2021-09-27 00:01:30,0,0,5664530014561852622,0,0,1240,0.035,-7.269,0,0.050,-5.370,1
1,2021-09-26 22:54:49,1,1,5186611064559013950,0,1,1,0.054,-2.657,269,0.032,-4.449,1
2,2021-09-26 23:57:20,2,2,2215519569292448030,0,0,2,0.014,-3.825,21,0.015,-3.939,1
3,2021-09-27 00:04:30,3,3,6262169206735077204,1,1,3,0.015,-3.461,99,0.051,-3.418,1
4,2021-09-27 00:06:21,4,4,4778985830203613115,1,0,4,0.051,-4.009,11464230,0.032,-2.829,1


Анализ для большей части датасета был выполнен еще в первом домашнем задании, посмотрим на наличие пропусков в новых данных:

In [21]:
data.isnull().sum()

date_time         0
zone_id           0
banner_id         0
oaid_hash         0
os_id             0
country_id        0
banner_id0        0
g0               69
coeff_sum0       69
banner_id1        0
g1            19744
coeff_sum1    19744
clicks            0
dtype: int64

Избавимся от таких наблюдений в тестовой части (так как они нужны для подсчета `ips`), в трейне нам такие наблюдения совершенно не помешают

**Добавим интеракции**: выбор обусловлен результатами, полученными в первом домашнем задании.

In [22]:
interactions = [
    ["banner_id", "country_id"],
    ["banner_id", "os_id"],
    ["banner_id", "zone_id"],
    ["banner_id", "hour"],
    ["banner_id", "weekday"],
]

**Информация по подготовке датасета:**

1.   Использована фича `oaid_hash`, сгруппированы редкие категории
2.   Было принято решение не группировать редкие категории для баннеров, так как непонятно что будет при подсчете `ips`
3. Редкие категории были сгруппированы только для фичи `zone_id`



In [29]:
(X_train, X_test, X_test1), (y_train, y_test, y_test), columns_for_ips = create_dataset(
    data, interactions
)

Dataset preparation...

Train size: 13692493
Test_size: 1885670
Features number: 4802697

Dataset is ready!


Обучаем линейную модель, как в первом домашнем задании с лучшими параметрами

In [30]:
model = LogisticRegression(solver="liblinear", penalty="l2", C=0.1).fit(
    X_train, y_train
)

In [31]:
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

In [34]:
y_model = model.predict_proba(X_test)[:, 1]
print(
    f"logLoss = {log_loss(y_test, y_model):.4f}, ROC AUC = {roc_auc_score(y_test, y_model):.4f}"
)

logLoss = 0.1316, ROC AUC = 0.8068


## CIPS

Необходимо получить вероятность того, что одна случайная величина с нормальным распределением имеет значение больше другой, давайте проделаем выкладки.

$\mathbb{P}(X > Y) = \mathbb{P}(X - Y > 0) = 1 - \mathbb{P}(X - Y <= 0) = 1 - F_{X-Y}(0)$ \

$X, Y$ - независимые случайные величины \

Пусть $X \sim N(m_1, d_1)$, $Y \sim N(m_2, d_2)$. Тогда $X-Y \sim N(m_1-m_2, \sqrt {d_1 ^ 2 + d_2^2})$

In [36]:
def get_policy_probability(coeffs0, g0, coeffs1, g1):
    mean = coeffs0 - coeffs1
    std = np.sqrt(g0**2 + g1**2) + 1e-9
    return 1 - norm.cdf(0, loc=mean, scale=std)


def get_cips(data_test, pi_0, pi_1, lambd=10):
    cips = np.mean(data_test * np.minimum(pi_1 / (pi_0 + 1e-9), lambd))
    print(f"CIPS: {cips:.4f}")

In [37]:
pi_0 = get_policy_probability(
    columns_for_ips["coeff_sum0"],
    columns_for_ips["g0"],
    columns_for_ips["coeff_sum1"],
    columns_for_ips["g1"],
)

y_pred0 = model.predict_proba(X_test)
y_pred1 = model.predict_proba(X_test1)
coeff_sum0_new = logit(y_pred0[:, 1])
coeff_sum1_new = logit(y_pred1[:, 1])

pi_1 = get_policy_probability(
    coeff_sum0_new, columns_for_ips["g0"], coeff_sum1_new, columns_for_ips["g1"]
)

In [43]:
get_cips(y_test, pi_0, pi_1)

CIPS: 0.0652
